In [1]:
#!/usr/bin/env python3
"""
THIRD SET COLAB 3: RL Training with Anthropic HH-RLHF
Uses Unsloth for efficient RL training
Dataset: Anthropic HH-RLHF (helpful/harmless)
Model: SmolLM2-135M
"""

print("="*80)
print("🚀 THIRD SET COLAB 3: RL TRAINING WITH ANTHROPIC HH-RLHF")
print("="*80)

# INSTALL
print("\n📦 Installing...")
import subprocess
subprocess.run("pip install -q unsloth", shell=True)
subprocess.run("pip uninstall -y unsloth-colab xformers-local flash-attn", shell=True)
print("✅ Done!")

# IMPORTS
print("\n📚 Importing...")
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import torch

print("✅ Imported!")
print(f"GPU: {torch.cuda.is_available()}")

# LOAD MODEL
print("\n📥 Loading model...")
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/SmolLM2-135M-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
print("✅ Loaded!")

# ADD LORA
print("\n🔧 Adding LoRA...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
print("✅ LoRA added!")

# LOAD ANTHROPIC HH-RLHF
print("\n📚 Loading Anthropic HH-RLHF dataset...")
dataset = load_dataset("Anthropic/hh-rlhf", split="train[:400]")
print(f"✅ Loaded {len(dataset)} preference examples!")

# FORMAT
print("\n🔧 Formatting...")

def format_rlhf(examples):
    texts = []
    for i in range(len(examples['chosen'])):
        # Use the chosen (preferred) responses for RL training
        chosen = examples['chosen'][i]
        # Clean up the format
        text = chosen.replace("\n\nHuman:", "\nUser:").replace("\n\nAssistant:", "\nAssistant:")
        texts.append(text + tokenizer.eos_token)
    return {"text": texts}

dataset = dataset.map(format_rlhf, batched=True, remove_columns=dataset.column_names)

print(f"✅ Formatted!")
print(f"\nExample:\n{dataset[0]['text'][:300]}...")

# TRAINING
print("\n🚀 RL training on preferred responses...")
training_args = TrainingArguments(
    output_dir="./rl_hhrlhf",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=5,
    logging_steps=10,
    save_strategy="no",
    fp16=True,
    report_to="none",
    max_steps=40,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

trainer.train()
print("\n✅ RL training complete!")

# INFERENCE
print("\n🧪 Testing RL model...")
FastLanguageModel.for_inference(model)

test_prompts = [
    "User: How can I improve my programming skills?\nAssistant: ",
    "User: What's a healthy breakfast?\nAssistant: ",
]

for prompt in test_prompts:
    print(f"\n{'='*60}")
    print(prompt.strip())
    print("Response:", end=" ")

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.7,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("Assistant:")[-1].strip()
    print(answer)

# SAVE
print("\n💾 Saving...")
model.save_pretrained("./hhrlhf_rl_model")
tokenizer.save_pretrained("./hhrlhf_rl_model")

print("\n" + "="*80)
print("🎉 THIRD SET COLAB 3 COMPLETE!")
print("="*80)
print("Summary:")
print("  ✓ Model: SmolLM2-135M")
print("  ✓ Dataset: Anthropic HH-RLHF (400 examples)")
print("  ✓ Method: RL on preferred responses")
print("="*80)

🚀 THIRD SET COLAB 3: RL TRAINING WITH ANTHROPIC HH-RLHF

📦 Installing...
✅ Done!

📚 Importing...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Imported!
GPU: True

📥 Loading model...
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/112M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

✅ Formatted!

Example:

User: What are some cuss words in english?
Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wan...

🚀 RL training on preferred responses...


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/400 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 1 | Total steps = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 1,843,200 of 136,358,784 (1.35% trained)


Step,Training Loss
10,2.711300
20,2.590500
30,2.591600
40,2.397900


Unsloth: Will smartly offload gradients to save VRAM!

✅ RL training complete!

🧪 Testing RL model...

User: How can I improve my programming skills?
Assistant:
Response: I want to learn more about programming.  I want to learn more about programming.  I want to learn more about programming.

User: What's a healthy breakfast?
Assistant:
Response: A healthy breakfast is a combination of foods that are high in protein and low in fat.  A healthy breakfast is a combination of foods that are high in protein and low in fat.  A healthy breakfast is a combination of foods that are high in protein and low in fat.  A healthy breakfast is a combination of foods that are high in protein and low in fat.

💾 Saving...

🎉 THIRD SET COLAB 3 COMPLETE!
Summary:
  ✓ Model: SmolLM2-135M
  ✓ Dataset: Anthropic HH-RLHF (400 examples)
  ✓ Method: RL on preferred responses
